## Setup and imports

In [1]:
import time
from pathlib import Path
import os
import numpy as np

import pandas as pd
from dotenv import load_dotenv
%load_ext autoreload
%autoreload 2
load_dotenv()
root = Path(os.getenv("ROOT"))
os.chdir(root)

## Test configuration parameters

In [2]:
layer_sizes = [1, 1, 2, 3]
activations = [1,300,400,2]
population_size = 1000
population_prev_times = np.full(population_size, int(time.time() * 1000))
prev_time = int(time.time() * 1000)
current_states = np.zeros(np.sum(layer_sizes[1:]), dtype=np.float64)
inputs = (prev_time, np.array([1.0]))
seed = 123

## Layer parameters calculation

In [3]:
from src.tann.network import get_layer_parameters
layer_parameters_count = get_layer_parameters(layer_sizes, layer_id=0)
%timeit -r 1 get_layer_parameters(layer_sizes, layer_id=0)

2.27 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 100,000 loops each)


## Total parameters calculation

In [4]:
from src.tann.network import get_total_parameters
total_parameters = get_total_parameters(layer_sizes)
%timeit -r 1 get_total_parameters(layer_sizes)

2.33 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 100,000 loops each)


## Activation function names

In [12]:
from src.tann.network.activation import get_activation_name
dict_activation = {i : get_activation_name(i) for i in range(10)}
print(dict_activation)
%timeit -r 1 [get_activation_name(i) for i in range(10)]

{0: 'linear', 1: 'relu', 2: 'sigmoid', 3: 'tanh', 4: 'leaky_relu', 5: 'unknown', 6: 'unknown', 7: 'unknown', 8: 'unknown', 9: 'unknown'}
701 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000,000 loops each)


## Benchmark activation functions performance

In [13]:
from src.tann.network.activation import apply_activation
import numpy as np
import time

value = 1.3324341

for i in range(10):
    times = []
    for _ in range(100000):
        start = time.perf_counter()
        apply_activation(value, activation_type=i)
        end = time.perf_counter()
        times.append((end - start) * 1e9)  # Convert to milliseconds
    print(
        f"Activation {i} - {get_activation_name(i)} - Time: {np.mean(times):.2f} ± {np.std(times):.2f} ns"
    )

Activation 0 - linear - Time: 162.13 ± 137.67 ns
Activation 1 - relu - Time: 128.02 ± 372.92 ns
Activation 2 - sigmoid - Time: 120.77 ± 40.84 ns
Activation 3 - tanh - Time: 121.36 ± 86.60 ns
Activation 4 - leaky_relu - Time: 115.10 ± 53.37 ns
Activation 5 - unknown - Time: 111.87 ± 43.38 ns
Activation 6 - unknown - Time: 116.49 ± 99.37 ns
Activation 7 - unknown - Time: 114.10 ± 27.89 ns
Activation 8 - unknown - Time: 115.58 ± 28.67 ns
Activation 9 - unknown - Time: 114.52 ± 24.33 ns


## Layer indices computation

In [14]:
from src.tann.network.utils import compute_layer_indices

param_indices, neuron_indices = compute_layer_indices(layer_sizes)
%timeit -r 1 compute_layer_indices(layer_sizes)

2.65 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 100,000 loops each)


## Individual initialization

In [15]:
from src.tann.evolution import initialize_individual
individual = initialize_individual(layer_sizes, seed=seed)
%timeit -r 1 individual = initialize_individual(layer_sizes, seed=seed)

3.54 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 100,000 loops each)


## Population initialization

In [16]:
from src.tann.evolution import initialize_population
population = initialize_population(population_size, layer_sizes, seed=seed)
%timeit -r 1 population = initialize_population(population_size, layer_sizes, seed=seed)

424 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)


## Individual prediction

In [17]:
from src.tann.evolution import predict_individual

current_values, new_states, current_time = predict_individual(individual, layer_sizes, activations, inputs, current_states, prev_time, param_indices, neuron_indices)

%timeit current_values, new_states, current_time = predict_individual(individual, layer_sizes, activations, inputs, current_states, prev_time, param_indices, neuron_indices)

5.34 μs ± 42.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Population prediction

In [18]:
from src.tann.evolution.population import predict_population
population_states = np.zeros((population_size, len(current_states)), dtype=np.float64)

outputs, updated_states, updated_times = predict_population(population, layer_sizes, activations, inputs, population_states, population_prev_times)

%timeit -r 1 outputs, updated_states, updated_times = predict_population(population, layer_sizes, activations, inputs, population_states, population_prev_times)

355 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
